In [ ]:
# Dowload files: 

# wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE243nnn/GSE243013/suppl/GSE243013_NSCLC_immune_scRNA_counts.mtx.gz

In [27]:
import gzip
import pandas as pd
import os
import re
import pandas as pd
import numpy as np
import scanpy as sc
import gseapy as gp
import seaborn as sns
import matplotlib.pyplot as plt
import anndata
import shutil
import pickle
import scipy.io
traget_rd = 3000

In [2]:
metadata = pd.read_csv("GSE243013_NSCLC_immune_scRNA_metadata.csv.gz")
print(metadata.shape)

metadata

/var/folders/fk/gk1pvdpx7fz79tx26t0rjc7c0000gp/T/ipykernel_81662/752780711.py:1: DtypeWarning: Columns (19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("GSE243013_NSCLC_immune_scRNA_metadata.csv.gz")


(1254749, 23)


,sampleID,cellID,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,major_cell_type,sub_cell_type,total_counts_rb,...,smoking_history,cancer_type,pre_treatment_staging,anti-PD1_therapy,chemotherapy,targeted_therapy,cycles,pathological_response,pathological_response_rate,radiological_response
0,P266,P266-ACACCAAAGGCACATG-1,1048,1048,2025.0,23.0,1.135803,T/NK cell,CD8T_Tem_GZMK+GZMH+,431.0,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
1,P266,P266-GGGCATCTCGTCACGG-1,827,827,1435.0,23.0,1.602788,T/NK cell,CD4T_Tm_ANXA1,198.0,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
2,P266,P266-GTGCTTCCATGATCCA-1,1254,1254,2441.0,81.0,3.318312,T/NK cell,CD4T_Tfh_CXCL13,554.0,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
3,P266,P266-GGAAAGCGTATAATGG-1,967,967,1602.0,38.0,2.372035,T/NK cell,CD8T_Tem_GZMK+GZMH+,200.0,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
4,P266,P266-CACAGTAAGCCCAGCT-1,1211,1211,2139.0,53.0,2.477794,T/NK cell,NK_CD16hi_FGFBP2,282.0,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254744,P159,P159-CATCCACCATGTTCCC-1,2192,2192,6506.0,239.0,3.673532,T/NK cell,NK_CD16hi_FGFBP2,1636.0,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm
1254745,P159,P159-TGTATTCGTATCGCAT-1,1869,1869,5544.0,57.0,1.028139,T/NK cell,CD4T_Tm_ANXA1,1580.0,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm
1254746,P159,P159-ATGCGATTCTTCGAGA-1,1940,1940,5715.0,68.0,1.189851,T/NK cell,CD4T_Tn_CCR7,1969.0,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm
1254747,P159,P159-CCAGCGATCGGTTAAC-1,2179,2179,5654.0,143.0,2.529183,T/NK cell,CD8T_Tm_IL7R,1367.0,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm


In [7]:
tcr = pd.read_csv("GSE243013_T_with_TCR_annotation.csv.gz")
print(tcr.shape)
print(tcr['sampleID'].value_counts().sort_values())
print(tcr.head().to_string())
print("Columns in tcr:", tcr.columns.tolist())

(434458, 15)
sampleID
P10       59
P248     136
P161     144
P296     149
P490     154
        ... 
P47     4736
P33     4895
P124    4895
P1      4910
P45     5099
Name: count, Length: 231, dtype: int64
  sampleID                   cellID        sub_cell_type TRA_v_gene TRA_j_gene TRA_c_gene        TRA_cdr3 TRB_v_gene TRB_j_gene TRB_c_gene           TRB_cdr3            clonotype     expansion  clonotype_number T_new_name
0     P481  P481-CTCACACGTATAGTAG-1      CD4T_Treg_FOXP3      TRAV6     TRAJ37       TRAC  CALDRSSNTGKLIF   TRBV20-1    TRBJ2-7      TRBC2      CSARTSGTYEQYF  P481_clonetype_1562  non-expanded                 1      other
1      P33   P33-AAAGCAACATCGGGTC-1       CD4T_Treg_CCR8   TRAV26-1     TRAJ42       TRAC    CIVRVGVGNLIF    TRBV6-1    TRBJ1-2      TRBC1    CASSESGFESYGYTF   P33_clonetype_1566  non-expanded                 1      other
2     P454  P454-TATTACCGTGACAAAT-1  CD8T_Tem_GZMK+GZMH+     TRAV19     TRAJ26       TRAC  CALSEWNYGQNFVF    TRBV6-5    TRBJ2-7   

In [10]:
# Read in TCR from individual sampel to check number of TCRs
tcr1 = pd.read_csv("GSE243013_RAW/P1_TCR/filtered_contig_annotations.csv.gz", compression='gzip')
print(tcr1.shape)
print(tcr1.head().to_string())
print("Number of unique barcodes:", tcr1['barcode'].nunique())
# Tabulate "productive"
print(tcr1['productive'].value_counts())

# Check unique barcodes when productive is True
productive_barcodes = tcr1.loc[tcr1['productive'], 'barcode'].unique()
print(f"Number of unique productive barcodes: {len(productive_barcodes)}")

# Filter for is_cell True, high_confidence True, productive True
filtered = tcr1[(tcr1['is_cell']) & (tcr1['high_confidence']) & (tcr1['productive'])]

# For TRA chain
tra_barcodes = filtered[filtered['chain'] == 'TRA']['barcode'].unique()
print(f"Number of unique barcodes for TRA: {len(tra_barcodes)}")

# For TRB chain
trb_barcodes = filtered[filtered['chain'] == 'TRB']['barcode'].unique()
print(f"Number of unique barcodes for TRB: {len(trb_barcodes)}")

# Intersection of TRA and TRB barcodes
intersection_barcodes = np.intersect1d(tra_barcodes, trb_barcodes)
print(f"Number of barcodes with both TRA and TRB: {len(intersection_barcodes)}")

(17529, 31)
              barcode  is_cell                    contig_id  high_confidence  length chain    v_gene d_gene   j_gene c_gene  full_length  productive                        fwr1                                                                         fwr1_nt    cdr1             cdr1_nt               fwr2                                              fwr2_nt     cdr2                cdr2_nt                                   fwr3                                                                                                          fwr3_nt              cdr3                                           cdr3_nt        fwr4                          fwr4_nt  reads  umis raw_clonotype_id           raw_consensus_id  exact_subclonotype_id
0  AAACCTGAGGCTCTTA-1     True  AAACCTGAGGCTCTTA-1_contig_1             True     502   TRB   TRBV3-1  TRBD1  TRBJ2-7  TRBC2         True        True  DTAVSQTPKYLVTQMGNDKSIKCEQN  GACACAGCTGTTTCCCAGACTCCAAAATACCTGGTCACACAGATGGGAAACGACAAGTCCATTAAATGTGAACAAA

The individual file as a bit more cell barcodes than the ones from "GSE243013_T_with_TCR_annotation.csv.gz". They probably have been filtered out for other reasons such as mitochondrial expression in scRNA-seq. Here we just use the cells from "GSE243013_T_with_TCR_annotation.csv.gz".

In [12]:
# Get cellIDs from tcr that start with 'P1-'
tcr_p1_cellIDs = tcr.loc[tcr['cellID'].str.startswith('P1-'), 'cellID']

# Check if 'P1-' + tcr1 barcodes are present in metadata cellID
tcr1_full_cellIDs = 'P1-' + intersection_barcodes

in_metadata = pd.Series(tcr1_full_cellIDs).isin(metadata['cellID'])

print(f"Number of tcr1 barcodes present in metadata: {in_metadata.sum()}")


Number of tcr1 barcodes present in metadata: 4913


In [14]:
not_in_tcr = set(tcr1_full_cellIDs) - set(tcr_p1_cellIDs)
print(f"Number of barcodes in tcr1 not in tcr: {len(not_in_tcr)}")
print("Example barcodes:", list(not_in_tcr)[:10])
print("Example records in tcr1 not in tcr:")


# Check if barcodes in not_in_tcr are present in metadata
not_in_tcr_in_metadata = [cell for cell in not_in_tcr if cell in metadata['cellID'].values]
print(f"Number of not_in_tcr barcodes present in cell metadata: {len(not_in_tcr_in_metadata)}")
print("Example barcodes in metadata:", not_in_tcr_in_metadata[:10])


Number of barcodes in tcr1 not in tcr: 1999
Example barcodes: ['P1-CATCAAGAGCCACGCT-1', 'P1-GAACGGAAGGCATGTG-1', 'P1-CCCAGTTCATTAGGCT-1', 'P1-AAAGTAGGTATAAACG-1', 'P1-TGAGCCGCATTCTCAT-1', 'P1-CATATGGTCGATCCCT-1', 'P1-ATCGAGTCAAGCGTAG-1', 'P1-AGATTGCGTCTTCGTC-1', 'P1-TAGGCATGTCCGAAGA-1', 'P1-AAAGATGTCCGAACGC-1']
Example records in tcr1 not in tcr:
Number of not_in_tcr barcodes present in cell metadata: 3
Example barcodes in metadata: ['P1-GGGAGATAGGACAGCT-1', 'P1-AATCGGTAGAGAACAG-1', 'P1-TAGAGCTTCGGAATCT-1']


In [15]:
# Select CD8T cells
cd8t_cells = metadata[
    (metadata['major_cell_type'] == 'T/NK cell') &
    (metadata['sub_cell_type'].str.startswith('CD8T'))
]

# Select CD4T cells
cd4t_cells = metadata[
    (metadata['major_cell_type'] == 'T/NK cell') &
    (metadata['sub_cell_type'].str.startswith('CD4T'))
]

print(f"CD8T cells: {cd8t_cells.shape[0]}")
print(f"CD4T cells: {cd4t_cells.shape[0]}")

CD8T cells: 348527
CD4T cells: 308073


In [16]:
metadata

,sampleID,cellID,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,major_cell_type,sub_cell_type,total_counts_rb,...,smoking_history,cancer_type,pre_treatment_staging,anti-PD1_therapy,chemotherapy,targeted_therapy,cycles,pathological_response,pathological_response_rate,radiological_response
0,P266,P266-ACACCAAAGGCACATG-1,1048,1048,2025.0,23.0,1.135803,T/NK cell,CD8T_Tem_GZMK+GZMH+,431.0,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
1,P266,P266-GGGCATCTCGTCACGG-1,827,827,1435.0,23.0,1.602788,T/NK cell,CD4T_Tm_ANXA1,198.0,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
2,P266,P266-GTGCTTCCATGATCCA-1,1254,1254,2441.0,81.0,3.318312,T/NK cell,CD4T_Tfh_CXCL13,554.0,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
3,P266,P266-GGAAAGCGTATAATGG-1,967,967,1602.0,38.0,2.372035,T/NK cell,CD8T_Tem_GZMK+GZMH+,200.0,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
4,P266,P266-CACAGTAAGCCCAGCT-1,1211,1211,2139.0,53.0,2.477794,T/NK cell,NK_CD16hi_FGFBP2,282.0,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254744,P159,P159-CATCCACCATGTTCCC-1,2192,2192,6506.0,239.0,3.673532,T/NK cell,NK_CD16hi_FGFBP2,1636.0,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm
1254745,P159,P159-TGTATTCGTATCGCAT-1,1869,1869,5544.0,57.0,1.028139,T/NK cell,CD4T_Tm_ANXA1,1580.0,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm
1254746,P159,P159-ATGCGATTCTTCGAGA-1,1940,1940,5715.0,68.0,1.189851,T/NK cell,CD4T_Tn_CCR7,1969.0,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm
1254747,P159,P159-CCAGCGATCGGTTAAC-1,2179,2179,5654.0,143.0,2.529183,T/NK cell,CD8T_Tm_IL7R,1367.0,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm


In [17]:
# Tabulate major_cell_type in metadata
major_counts = metadata['major_cell_type'].value_counts()
print("Major cell type counts:")
print(major_counts)

# Tabulate sub_cell_type among 'T/NK cell'
tnk_sub_counts = metadata.loc[metadata['major_cell_type'] == 'T/NK cell', 'sub_cell_type'].value_counts()
print("\nSub cell type counts within 'T/NK cell':")
print(tnk_sub_counts)

Major cell type counts:
major_cell_type
T/NK cell       766574
B cell          297076
Myeloid cell    191099
Name: count, dtype: int64

Sub cell type counts within 'T/NK cell':
sub_cell_type
CD4T_Tn_CCR7              70896
CD4T_Tm_ANXA1             67089
CD8T_Tm_IL7R              65845
CD8T_Tem_GZMK+GZMH+       60731
CD8T_Tem_GZMK+NR4A1+      57363
CD4T_Treg_FOXP3           53900
CD8T_Tex_CXCL13           45499
CD8T_Trm_ZNF683           42240
NK_CD16hi_FGFBP2          40286
CD4T_Tem_GZMA             33453
CD4T_Tfh_CXCL13           31642
CD8T_NK-like_FGFBP2       30002
NK_CD16low_GZMK           28551
CD4T_Treg_CCR8            24492
T_gdT_TRDV1               23517
CD4T_Th1-like_CXCL13      18260
CD8T_terminal_Tex_LAYN    14225
CD8T_prf_MKI67            12268
CD8T_MAIT_KLRB1           11728
T_gdT_TRDV2               10435
CD8T_ISG15                 8626
ILC3_KIT                   7185
CD4T_Tm_XCL1               6018
CD4T_Treg_MKI67            2323
Name: count, dtype: int64


In [18]:
barcodes = pd.read_csv("GSE243013_barcodes.csv.gz")
print(barcodes.shape)
# Check if the barcodes and metadata.cellID are identical and in the same order
same_order = (barcodes['barcode'].values == metadata['cellID'].values).all()
print(f"Barcodes and metadata.cellID are identical and in the same order: {same_order}")

(1254749, 1)
Barcodes and metadata.cellID are identical and in the same order: True


In [19]:
features = pd.read_csv("GSE243013_genes.csv.gz")
print(features.shape)
features.head()

(31831, 1)


,geneSymbol
0,MIR1302-2HG
1,AL627309.1
2,AL627309.3
3,AL627309.2
4,AL627309.4


In [20]:
X = scipy.io.mmread("GSE243013_NSCLC_immune_scRNA_counts.mtx.gz").tocsr()
print(X.shape)

(1254749, 31831)


In [21]:
# Set feature names (row names)
var = pd.DataFrame(index=features['geneSymbol'].tolist())
obs = pd.DataFrame(index=barcodes['barcode'].tolist())

# Create AnnData
adata = anndata.AnnData(X=X, obs=obs, var=var)
adata

adata.var["gene_name"] = features["geneSymbol"].values

In [23]:
import pickle

with open("signatures_CD8.pkl", "rb") as f:
    sigs_CD8 = pickle.load(f)


with open("signatures_CD4.pkl", "rb") as f:
    sigs_CD4 = pickle.load(f)

del sigs_CD4['Jansen_TermDiff_73g']
del sigs_CD8['Lowery_neg_99g']

print({k: len(v) for k, v in sigs_CD8.items()})
print({k: len(v) for k, v in sigs_CD4.items()})

def average_genes(genes_list, df):
    valid_genes = [g for g in genes_list if g in df.columns]
    if len(valid_genes) == 0:
        return np.repeat(np.nan, df.shape[0])
    sub_df = df[valid_genes]
    return sub_df.mean(axis=1)  # mean over columns => per-cell average

def match_genes_in_sets(sigs_CD8, se2):    
    # Dictionary to hold matched genes for each set
    gs = {}
    # Loop over each key in the sigs_CD8 dictionary
    for s1 in sigs_CD8.keys():
        # 'sigs_CD8[s1]' is a list of genes
        # We'll find which of these genes appear in se2.index
        matched_genes = []
        
        for gene in sigs_CD8[s1]:
            if gene in se2['gene_name'].tolist():
                matched_genes.append(gene)
        
        # Print match info
        print(f"{s1}: {len(matched_genes)}/{len(sigs_CD8[s1])} genes are found.")
        
        # Store in gs
        gs[s1] = matched_genes
    return

{'Lowery_pos_243g': 243, 'Oliveira_TTE_100g': 100, 'Oliveira_pos_74g': 74, 'Yost_CD8_Exh_100g': 100, 'Hanada_pos_27g': 27, 'Oliveira_virus_26g': 26, 'Hanada_neg_5g': 5}
{'Lowery_pos_40g': 40, 'Caushi_Tfh2_66g': 66, 'Oh_CXCL13_50g': 50, 'Hanada_pos_9g': 9, 'Lowery_neg_37g': 37, 'Hanada_neg_4g': 4}


In [24]:
final_df = pd.DataFrame()
t_cell = {"CD4":[
'CD4T_Tem_GZMA', 
'CD4T_Tfh_CXCL13', 
'CD4T_Th1-like_CXCL13', 
'CD4T_Tm_ANXA1', 
'CD4T_Tm_XCL1', 
'CD4T_Tn_CCR7', 
'CD4T_Treg_CCR8', 
'CD4T_Treg_FOXP3', 
'CD4T_Treg_MKI67'], 
"CD8":[
'CD8T_ISG15', 
'CD8T_MAIT_KLRB1', 
'CD8T_NK-like_FGFBP2', 
'CD8T_prf_MKI67', 
'CD8T_Tem_GZMK+GZMH+', 
'CD8T_Tem_GZMK+NR4A1+', 
'CD8T_terminal_Tex_LAYN', 
'CD8T_Tex_CXCL13', 
'CD8T_Tm_IL7R', 
'CD8T_Trm_ZNF683'
]}
t_cell

{'CD4': ['CD4T_Tem_GZMA',
  'CD4T_Tfh_CXCL13',
  'CD4T_Th1-like_CXCL13',
  'CD4T_Tm_ANXA1',
  'CD4T_Tm_XCL1',
  'CD4T_Tn_CCR7',
  'CD4T_Treg_CCR8',
  'CD4T_Treg_FOXP3',
  'CD4T_Treg_MKI67'],
 'CD8': ['CD8T_ISG15',
  'CD8T_MAIT_KLRB1',
  'CD8T_NK-like_FGFBP2',
  'CD8T_prf_MKI67',
  'CD8T_Tem_GZMK+GZMH+',
  'CD8T_Tem_GZMK+NR4A1+',
  'CD8T_terminal_Tex_LAYN',
  'CD8T_Tex_CXCL13',
  'CD8T_Tm_IL7R',
  'CD8T_Trm_ZNF683']}

In [25]:
adata.obs = adata.obs.join(metadata.set_index('cellID'))
adata.obs

,sampleID,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,major_cell_type,sub_cell_type,total_counts_rb,pct_counts_rb,...,smoking_history,cancer_type,pre_treatment_staging,anti-PD1_therapy,chemotherapy,targeted_therapy,cycles,pathological_response,pathological_response_rate,radiological_response
P266-ACACCAAAGGCACATG-1,P266,1048,1048,2025.0,23.0,1.135803,T/NK cell,CD8T_Tem_GZMK+GZMH+,431.0,21.283950,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
P266-GGGCATCTCGTCACGG-1,P266,827,827,1435.0,23.0,1.602788,T/NK cell,CD4T_Tm_ANXA1,198.0,13.797909,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
P266-GTGCTTCCATGATCCA-1,P266,1254,1254,2441.0,81.0,3.318312,T/NK cell,CD4T_Tfh_CXCL13,554.0,22.695616,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
P266-GGAAAGCGTATAATGG-1,P266,967,967,1602.0,38.0,2.372035,T/NK cell,CD8T_Tem_GZMK+GZMH+,200.0,12.484395,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
P266-CACAGTAAGCCCAGCT-1,P266,1211,1211,2139.0,53.0,2.477794,T/NK cell,NK_CD16hi_FGFBP2,282.0,13.183731,...,Y,LUAD,IIB,Camrelizumab,Carboplatin + Abraxane,No,3,non-MPR,0.05,SD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P159-CATCCACCATGTTCCC-1,P159,2192,2192,6506.0,239.0,3.673532,T/NK cell,NK_CD16hi_FGFBP2,1636.0,25.146019,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm
P159-TGTATTCGTATCGCAT-1,P159,1869,1869,5544.0,57.0,1.028139,T/NK cell,CD4T_Tm_ANXA1,1580.0,28.499279,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm
P159-ATGCGATTCTTCGAGA-1,P159,1940,1940,5715.0,68.0,1.189851,T/NK cell,CD4T_Tn_CCR7,1969.0,34.453194,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm
P159-CCAGCGATCGGTTAAC-1,P159,2179,2179,5654.0,143.0,2.529183,T/NK cell,CD8T_Tm_IL7R,1367.0,24.177574,...,Y,LUSC,IIIB,Pembrolizumab,Cisplatin + Abraxane,No,unknowm,MPR,1.0,unknowm


In [26]:
adata_all_CD4 = adata[adata.obs["sub_cell_type"].isin(t_cell['CD4'])]
adata_all_CD8 = adata[adata.obs["sub_cell_type"].isin(t_cell['CD8'])]

print(f"CD4T cells: {adata_all_CD4.shape[0]}")
print(f"CD8T cells: {adata_all_CD8.shape[0]}")

CD4T cells: 308073
CD8T cells: 348527


In [28]:
sc.pp.normalize_total(adata_all_CD4, target_sum=traget_rd)
sc.pp.log1p(adata_all_CD4)  # log transform
median_depth = np.median(adata_all_CD4.obs['total_counts'].values)
print("CD4T cells median depth.", median_depth)

sc.pp.normalize_total(adata_all_CD8, target_sum=traget_rd)
sc.pp.log1p(adata_all_CD8)  # log transform
median_depth = np.median(adata_all_CD8.obs['total_counts'].values)
print("CD8T cells median depth.", median_depth)


/Users/wsun/anaconda3/envs/CAT/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


CD4T cells median depth. 3233.0
CD8T cells median depth. 3115.0


Try one cell type as an exploration

In [29]:
lib = 'CD4T_Treg_MKI67'
print(lib)
adata = adata_all_CD4[adata_all_CD4.obs["sub_cell_type"]==lib]
print(adata)
print(adata.var.head())

CD4T_Treg_MKI67
View of AnnData object with n_obs × n_vars = 2323 × 31831
    obs: 'sampleID', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'major_cell_type', 'sub_cell_type', 'total_counts_rb', 'pct_counts_rb', 'gender', 'age', 'smoking_history', 'cancer_type', 'pre_treatment_staging', 'anti-PD1_therapy', 'chemotherapy', 'targeted_therapy', 'cycles', 'pathological_response', 'pathological_response_rate', 'radiological_response'
    var: 'gene_name'
    uns: 'log1p'
               gene_name
MIR1302-2HG  MIR1302-2HG
AL627309.1    AL627309.1
AL627309.3    AL627309.3
AL627309.2    AL627309.2
AL627309.4    AL627309.4


In [34]:
sets_ave_CD4 = ["Hanada_pos_9g", "Hanada_neg_4g"]
# Identify the rest for ssGSEA
ssgsea_sets = {k: v for k, v in sigs_CD4.items() if k not in sets_ave_CD4}

all_cells = np.array(adata.obs.index)
n_batches = 10
batch_size = len(all_cells) // n_batches
batches = np.array_split(all_cells, n_batches)

# Store results
ssgsea_all_results = []

ave_Hanada_pos_9g = []
ave_Hanada_neg_4g = []

for i, batch_cells in enumerate(batches):
    print(f"Processing batch {i+1} / {n_batches} ...")

    # Extract gene expression matrix for this batch
    expr_batch = pd.DataFrame(
        adata[batch_cells].X.transpose().toarray(),  # genes x cells
        index=adata.var.index,
        columns=batch_cells
    )

    # Run ssGSEA
    result = gp.ssgsea(data=expr_batch,
                       gene_sets=ssgsea_sets,
                       sample_norm=False,
                       outdir=None,
                       min_size=1,
                       max_size=20000,
                       permutation_num=0)
    result.run()

    # Store result
    ssgsea_all_results.append(result.res2d)


    expr_batch = pd.DataFrame(
    adata[batch_cells].X.toarray(), 
    index=batch_cells, 
    columns=adata.var.index)

    ave_Hanada_pos_9g = np.concatenate([ave_Hanada_pos_9g, 
                                        average_genes(sigs_CD4["Hanada_pos_9g"], expr_batch)])
    
    ave_Hanada_neg_4g = np.concatenate([ave_Hanada_neg_4g, 
                                        average_genes(sigs_CD4["Hanada_neg_4g"], expr_batch)])



# Combine all batches into one DataFrame


Processing batch 1 / 10 ...
Processing batch 2 / 10 ...
Processing batch 3 / 10 ...
Processing batch 4 / 10 ...
Processing batch 5 / 10 ...
Processing batch 6 / 10 ...
Processing batch 7 / 10 ...
Processing batch 8 / 10 ...
Processing batch 9 / 10 ...
Processing batch 10 / 10 ...


In [35]:
expr_batch.shape

(232, 31831)

In [37]:

print(ave_Hanada_pos_9g.size)
print(ave_Hanada_neg_4g.size)
print(ave_Hanada_pos_9g)
print(ave_Hanada_neg_4g)


2323
2323
[0.37302546 0.96129738 0.69590444 ... 0.76473583 0.57608223 0.77635012]
[1.80365323 1.91973347 1.57842963 ... 2.03426665 1.47525248 2.03651906]


In [38]:
combined_ssgsea = pd.concat(ssgsea_all_results, axis=0)
combined_ssgsea


,Name,Term,ES,NES
0,P432-TGCCCTAGTTCCGTCT-1,Lowery_neg_37g,13790.075813,0.801735
1,P59-GATCGCGTCCGTACAA-1,Lowery_neg_37g,13499.054248,0.784815
2,P5-CTGTTTATCCAAAGTC-1,Lowery_neg_37g,13159.650049,0.765083
3,P36-TAGCCGGAGGAGTACC-1,Lowery_neg_37g,13078.313543,0.760354
4,P67-CGAACATGTCTTGCGG-1,Lowery_neg_37g,12978.743136,0.754565
...,...,...,...,...
923,P416-GAAGCAGCAGGACGTA-1,Lowery_pos_40g,-16.618062,-0.000993
924,P120-GAACGGAGTTCGTTGA-1,Lowery_pos_40g,-15.024636,-0.000898
925,P527-GCAATCACAATCGGTT-1,Oh_CXCL13_50g,11.154824,0.000667
926,P39-GTCTCGTAGCCAACAG-1,Oh_CXCL13_50g,-3.52714,-0.000211


In [39]:

# Transpose to cell x set
df_wide = combined_ssgsea.pivot(
    index='Name',    # Each sample’s name/ID
    columns='Term',  # The gene set name
    values='NES'      # Which score you want to spread out as columns
)
# rename columns
df_wide.columns = [f"CD4{col}" for col in df_wide.columns]

df_wide

,CD4Caushi_Tfh2_66g,CD4Lowery_neg_37g,CD4Lowery_pos_40g,CD4Oh_CXCL13_50g
Name,,,,
P1-AACTCCCTCACAGTAC-1,0.419851,0.604766,0.500681,0.071455
P1-AAGTCTGAGCGTGTCC-1,0.237827,0.373066,0.273399,0.023549
P1-ACCGTAAGTCGAGTTT-1,0.282048,0.494304,0.43621,0.060797
P1-ACTGCTCCAGATAATG-1,0.433091,0.532682,0.43533,-0.021036
P1-ACTTGTTGTCACCCAG-1,0.289317,0.567804,0.354367,-0.00418
...,...,...,...,...
P97-CTGATCCAGGCGTACA-1,0.358054,0.693391,0.598996,0.073313
P97-GACAGAGAGCTAACAA-1,0.339381,0.606374,0.576605,0.055835
P97-GCACATAAGCCACCTG-1,0.251208,0.59772,0.496294,0.114115


In [40]:
# Create a DataFrame for the average signatures
signature_df = pd.DataFrame(index=adata.obs.index)
signature_df["Hanada_pos_9g"] = ave_Hanada_pos_9g
signature_df["Hanada_neg_4g"] = ave_Hanada_neg_4g

df_combined = df_wide.join(signature_df, how="inner")
df_combined

,CD4Caushi_Tfh2_66g,CD4Lowery_neg_37g,CD4Lowery_pos_40g,CD4Oh_CXCL13_50g,Hanada_pos_9g,Hanada_neg_4g
P1-AACTCCCTCACAGTAC-1,0.419851,0.604766,0.500681,0.071455,0.658974,1.774364
P1-AAGTCTGAGCGTGTCC-1,0.237827,0.373066,0.273399,0.023549,0.705200,2.280715
P1-ACCGTAAGTCGAGTTT-1,0.282048,0.494304,0.43621,0.060797,0.713842,1.519644
P1-ACTGCTCCAGATAATG-1,0.433091,0.532682,0.43533,-0.021036,0.718508,1.207381
P1-ACTTGTTGTCACCCAG-1,0.289317,0.567804,0.354367,-0.00418,0.578935,2.342358
...,...,...,...,...,...,...
P97-CTGATCCAGGCGTACA-1,0.358054,0.693391,0.598996,0.073313,0.882759,2.081815
P97-GACAGAGAGCTAACAA-1,0.339381,0.606374,0.576605,0.055835,0.671875,1.876100
P97-GCACATAAGCCACCTG-1,0.251208,0.59772,0.496294,0.114115,0.843240,2.385302
P97-GCCAAATCAAACGCGA-1,0.396982,0.686746,0.541741,-0.003051,0.738618,2.111792


In [44]:
final_df_CD4 = pd.DataFrame()
final_df_CD4 = pd.concat([final_df_CD4, df_combined], ignore_index=False)
final_df_CD4

,CD4Caushi_Tfh2_66g,CD4Lowery_neg_37g,CD4Lowery_pos_40g,CD4Oh_CXCL13_50g,Hanada_pos_9g,Hanada_neg_4g
P1-AACTCCCTCACAGTAC-1,0.419851,0.604766,0.500681,0.071455,0.658974,1.774364
P1-AAGTCTGAGCGTGTCC-1,0.237827,0.373066,0.273399,0.023549,0.705200,2.280715
P1-ACCGTAAGTCGAGTTT-1,0.282048,0.494304,0.43621,0.060797,0.713842,1.519644
P1-ACTGCTCCAGATAATG-1,0.433091,0.532682,0.43533,-0.021036,0.718508,1.207381
P1-ACTTGTTGTCACCCAG-1,0.289317,0.567804,0.354367,-0.00418,0.578935,2.342358
...,...,...,...,...,...,...
P97-CTGATCCAGGCGTACA-1,0.358054,0.693391,0.598996,0.073313,0.882759,2.081815
P97-GACAGAGAGCTAACAA-1,0.339381,0.606374,0.576605,0.055835,0.671875,1.876100
P97-GCACATAAGCCACCTG-1,0.251208,0.59772,0.496294,0.114115,0.843240,2.385302
P97-GCCAAATCAAACGCGA-1,0.396982,0.686746,0.541741,-0.003051,0.738618,2.111792
